In [ ]:
%matplotlib inline

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 7

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup 

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Monte Carlo simulation of the Ising model

## Metropolis simulation

First we'll define a function which performs the Metropolis algorithm. Note that we are taking $J = 1$.

In [ ]:
def metropolis(
    T: float,
    L: int,
    num_equilibrium_sweeps: int = 200,
    num_sample_sweeps: int = 50,
    num_samples: int = 300,
) -> np.ndarray:
    """Perform Metropolis algorithm

    Returns a 2 x num_samples array, where the
    first row contains energy values for each sample and
    the second row contains magnitization values for each
    sample.
    """
    # Set up a bunch of convenience functions
    left = lambda pos: (pos[0], pos[1] - 1)
    right = lambda pos: (pos[0], (pos[1] + 1) % L)
    top = lambda pos: (pos[0] - 1, pos[1])
    bottom = lambda pos: ((pos[0] + 1) % L, pos[1])

    random_pos = lambda: tuple(np.random.randint(L, size=2))

    def get_total_energy() -> float:
        """Get total energy of the lattice."""
        total = 0

        for pos in itertools.product(range(L), range(L)):
            total += (
                lattice[pos] * lattice[left(pos)]
                + lattice[pos] * lattice[right(pos)]
                + lattice[pos] * lattice[top(pos)]
                + lattice[pos] * lattice[bottom(pos)]
            )

        return -0.5 * total

    def get_change_in_energy(pos) -> float:
        """Get change in energy due to flipping one spin."""
        pos_total = (
            lattice[pos] * lattice[left(pos)]
            + lattice[pos] * lattice[right(pos)]
            + lattice[pos] * lattice[top(pos)]
            + lattice[pos] * lattice[bottom(pos)]
        )

        return 2 * pos_total

    def get_total_magnetization() -> int:
        """Get the total magnetization of the lattice.
        
        Note that we don't bother with changes in
        magnetization (like we do with energy) since this
        calculation is very efficient.
        """
        return np.sum(lattice)

    def change_is_accepted(dE: float) -> bool:
        """Determine if the change is accepted."""
        return np.exp(-dE / T) >= np.random.rand()

    def perform_sweep() -> None:
        """Randomly flip spins L^2 times according to Boltzmann weights."""
        nonlocal energy, lattice

        for _ in range(L ** 2):
            # Develop a candidate flip
            pos = random_pos()
            dE = get_change_in_energy(pos)

            # If accepted, make the flip
            if change_is_accepted(dE):
                energy += dE
                lattice[pos] = -lattice[pos]

    # Initialize the lattice with random spin orientations
    lattice = np.random.choice([-1, 1], size=(L, L))
    energy = get_total_energy()

    # Perform equilibrium sweeps
    for _ in range(num_equilibrium_sweeps):
        perform_sweep()

    # Now take samples
    data = np.zeros((2, num_samples))
    data[:, 0] = [energy, get_total_magnetization()]

    for i in range(1, num_samples):
        # Sweep a number of times
        for _ in range(num_sample_sweeps):
            perform_sweep()

        data[:, i] = [energy, get_total_magnetization()]

    # Return data
    return data

## Data generation

We will consider data for simulations using the following values of temperatures $T$ and lattice lengths $L$.

In [ ]:
Ts = np.array([1.5, 2.0, 2.1, 2.2, 2.25, 2.26, 2.27, 2.28, 2.29, 2.3, 2.5, 3.0])
Ls = np.array([10, 15, 20, 25])

Now we'll either generate data **OR** load data we already have saved.

In [ ]:
load_existing_data = True

Now we'll generate the data (if the above option is `False`). Each simulations data will be saved to a file `Lxx_Txxx.npy` file where an example filename would be `L25_T250.npy` which corresponds to $L = 25, T = 2.50$.

In [ ]:
if not load_existing_data:
    for T, L in itertools.product(Ts, Ls):
        data = metropolis(T, L)

        filename = "L%02d_T%03d.npy" % (L, int(round(T * 100)))
        np.save(filename, data)

## Analysis

Now that we have the data we need, let's compute the ensemble averages.

In [ ]:
average_abs_Ms = np.zeros((len(Ls), len(Ts)))
average_M_squareds = np.zeros((len(Ls), len(Ts)))
average_M_fourth_powers = np.zeros((len(Ls), len(Ts)))
average_Es = np.zeros((len(Ls), len(Ts)))
average_E_squareds = np.zeros((len(Ls), len(Ts)))

In [ ]:
for idxL, L in enumerate(Ls):
    for idxT, T in enumerate(Ts):
        # Load data
        filename = "L%02d_T%03d.npy" % (L, int(round(T * 100)))
        loaded_data = np.load(filename)

        # Add ensemble averages
        average_abs_Ms[idxL, idxT] = np.mean(np.abs(loaded_data[1, :]))
        average_M_squareds[idxL, idxT] = np.mean(loaded_data[1, :] ** 2)
        average_M_fourth_powers[idxL, idxT] = np.mean(loaded_data[1, :] ** 4)
        average_Es[idxL, idxT] = np.mean(loaded_data[0, :])
        average_E_squareds[idxL, idxT] = np.mean(loaded_data[0, :] ** 2)

Now we will produce a plot of the mean absolute magnetization per spin for our different values of $L$.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
for idxL, L in enumerate(Ls):
    plt.plot(Ts, average_abs_Ms[idxL, :] / L ** 2, "-+")

# Labels
ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$\langle |M| \rangle / N$")

ax.legend(["L = %s" % L for L in Ls]);

**DESCRIBE ABOVE PLOT**

Now we'll plot the specific heat per spin.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
for idxL, L in enumerate(Ls):
    plt.plot(
        Ts,
        (average_E_squareds[idxL, :] - average_Es[idxL, :] ** 2) / (Ts ** 2 * L ** 2),
        "-+",
    )

# Labels
ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$C_V / N k_B$")

ax.legend(["L = %s" % L for L in Ls]);

**DESCRIBE ABOVE PLOT**

Now we'll plot the susceptibility per spin.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
for idxL, L in enumerate(Ls):
    plt.plot(
        Ts,
        (average_M_squareds[idxL, :] - average_abs_Ms[idxL, :] ** 2) / (Ts * L ** 2),
        "-+",
    )

# Labels
ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$\chi_M / N$")

ax.legend(["L = %s" % L for L in Ls]);

**DESCRIBE ABOVE PLOT**

Now we'll plot the Binder cumulant $g$.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
for idxL, L in enumerate(Ls):
    plt.plot(
        Ts,
        3
        / 2
        * (
            1
            - average_M_fourth_powers[idxL, :] / (3 * average_M_squareds[idxL, :] ** 2)
        ),
        "-+",
    )

# Labels
ax.set_xlabel(r"$T$")
ax.set_ylabel(r"$g$")

ax.legend(["L = %s" % L for L in Ls]);

**DESCRIBE ABOVE PLOT**